In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import plotly.graph_objects as go

from plotly import 
import numpy as np
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
# Read csv and save to parquet. Needs to be done only once
# transactions = pd.read_csv('data/synt_transactions_10M.csv', sep=';')
# transactions.to_parquet('data/transactions_categorical.parquet')

In [5]:
transactions = pd.read_parquet('data/transactions_categorical.parquet')
display(transactions)


,category,tstamp,arvopvm,kirjauspvm,maksupvm,tilinro,rahamaara,saldo,vientiselitekd,taplajikd,bic_saaja,viite,iban_saaja,counterparty_account_id
0,Tulot,2019-01-01 00:00:00.000000000,2019-01-01,2019-01-01,2019-01-01,1.0,22.09,59.55,0.0,506.0,,1.0,1.0,177.0
1,Ruoka_paivittaistavarakauppa,2019-01-01 13:54:23.551401869,2019-01-01,2019-01-02,2019-01-01,1.0,-9.47,50.08,0.0,162.0,,1.0,1.0,116.0
2,Ruoka_paivittaistavarakauppa,2019-01-02 03:48:47.102803738,2019-01-02,2019-01-02,2019-01-02,1.0,-16.77,33.31,0.0,162.0,,1.0,1.0,137.0
3,Ruoka_paivittaistavarakauppa,2019-01-02 17:43:10.654205607,2019-01-02,2019-01-02,2019-01-02,1.0,-0.89,32.42,0.0,162.0,,1.0,1.0,33.0
4,Ruoka_paivittaistavarakauppa,2019-01-03 07:37:34.205607476,2019-01-03,2019-01-03,2019-01-03,1.0,-5.93,26.49,0.0,162.0,,1.0,1.0,125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9936641,Asuminen,2019-07-05 10:01:09.902912619,2019-07-05,2019-07-05,2019-07-05,9155.0,-26.21,935.10,706.0,129.0,OKOYFIHH,0.0,0.0,234.0
9936642,Ravintolat_kahvilat,2019-07-05 19:34:22.135922328,2019-07-05,2019-07-05,2019-07-05,9155.0,-11.84,923.26,0.0,162.0,,1.0,1.0,43.0
9936643,Tulot,2019-07-08 05:21:33.203883492,2019-07-08,2019-07-08,2019-07-08,9155.0,124.01,1047.27,710.0,588.0,,0.0,1.0,252.0
9936644,Shoppailu,2019-07-08 14:54:45.436893201,2019-07-08,2019-07-08,2019-07-08,9155.0,-17.80,1029.47,0.0,162.0,,1.0,1.0,128.0


,category,tstamp,arvopvm,kirjauspvm,maksupvm,tilinro,rahamaara,saldo,vientiselitekd,taplajikd,bic_saaja,viite,iban_saaja,counterparty_account_id
0,Tulot,2019-01-01 00:00:00.000000000,2019-01-01,2019-01-01,2019-01-01,1.0,22.09,59.55,0.0,506.0,,1.0,1.0,177.0
1,Ruoka_paivittaistavarakauppa,2019-01-01 13:54:23.551401869,2019-01-01,2019-01-02,2019-01-01,1.0,-9.47,50.08,0.0,162.0,,1.0,1.0,116.0
2,Ruoka_paivittaistavarakauppa,2019-01-02 03:48:47.102803738,2019-01-02,2019-01-02,2019-01-02,1.0,-16.77,33.31,0.0,162.0,,1.0,1.0,137.0
3,Ruoka_paivittaistavarakauppa,2019-01-02 17:43:10.654205607,2019-01-02,2019-01-02,2019-01-02,1.0,-0.89,32.42,0.0,162.0,,1.0,1.0,33.0
4,Ruoka_paivittaistavarakauppa,2019-01-03 07:37:34.205607476,2019-01-03,2019-01-03,2019-01-03,1.0,-5.93,26.49,0.0,162.0,,1.0,1.0,125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9936641,Asuminen,2019-07-05 10:01:09.902912619,2019-07-05,2019-07-05,2019-07-05,9155.0,-26.21,935.10,706.0,129.0,OKOYFIHH,0.0,0.0,234.0
9936642,Ravintolat_kahvilat,2019-07-05 19:34:22.135922328,2019-07-05,2019-07-05,2019-07-05,9155.0,-11.84,923.26,0.0,162.0,,1.0,1.0,43.0
9936643,Tulot,2019-07-08 05:21:33.203883492,2019-07-08,2019-07-08,2019-07-08,9155.0,124.01,1047.27,710.0,588.0,,0.0,1.0,252.0
9936644,Shoppailu,2019-07-08 14:54:45.436893201,2019-07-08,2019-07-08,2019-07-08,9155.0,-17.80,1029.47,0.0,162.0,,1.0,1.0,128.0


In [31]:
date_columns = ['tstamp', 'arvopvm', 'kirjauspvm', 'maksupvm']
transactions = transactions.dropna()
for column in date_columns:
    transactions[column] = pd.to_datetime(transactions[column])
incomes = transactions.loc[transactions['category'] == 'Tulot']
account_grouped = transactions.groupby('tilinro')
min_dates = account_grouped['maksupvm'].min()
max_dates = account_grouped['maksupvm'].max()
date_deltas = max_dates - min_dates
monthly_incomes = transactions.loc[transactions['category'] == 'Tulot'].groupby('tilinro')['rahamaara'].sum() / date_deltas.dt.days * 365 / 12
monthly_categories = pd.DataFrame()
for category_name in transactions['category'].unique():
    monthly_categories[category_name] = transactions.loc[transactions['category'] == category_name].groupby('tilinro')['rahamaara'].sum() / date_deltas.dt.days * 365 / 12
living_costs = -transactions.loc[transactions['category'] == 'Asuminen'].groupby('tilinro')['rahamaara'].sum() / date_deltas.dt.days * 365 / 127
living_costs = -transactions.loc[transactions['category'] == 'Asuminen'].groupby('tilinro')['rahamaara'].sum() / date_deltas.dt.days * 365 / 12

display(living_costs)
monthly_categories.to_parquet('data/monthly_categories.parquet')

tilinro
1.0         37.969930
2.0        290.620833
3.0       1270.659583
4.0        263.995417
5.0        128.022500
             ...     
9151.0     358.481250
9152.0     117.857083
9153.0     212.798171
9154.0     268.432083
9155.0     158.569526
Length: 9155, dtype: float64

In [25]:
transactions.to_parquet('data/transactions.parquet', engine='fastparquet', compression='gzip')

In [26]:
example = transactions.loc[transactions['tilinro'] == 1]

monthly_incomes.to_csv('monthly_incomes.csv')

In [27]:
example.loc[:, 'maksupvm'] = pd.to_datetime(example['maksupvm'])
transactions = transactions.dropna()

transactions.loc[:, 'maksupvm'] = pd.to_datetime(transactions['maksupvm'])

# print((example['maksupvm'].dt.month == 2) and (example['maksupvm'].dt.year == 2020))
month = example.loc[(example['maksupvm'].dt.month == 2) & (example['maksupvm'].dt.year == 2020)]



/home/juho/.pyenv/versions/3.8.6/envs/junction_2020/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [28]:
display(pd.Series(transactions['category'].unique()))

0                            Tulot
1     Ruoka_paivittaistavarakauppa
2                      Liikkuminen
3                 Kulttuuri_viihde
4                             Muut
5                         Asuminen
6               Luokittelemattomat
7                         Matkailu
8                 Luottojen_maksut
9              Saastot_sijoitukset
10             Ravintolat_kahvilat
11                     Vakuutukset
12                         Terveys
13                    Harrastukset
14            Hyvinvointijakauneus
15                        Lemmikit
16                     Hyvinvointi
17                          Lapset
18                       Shoppailu
dtype: object

In [30]:
%matplotlib widget
plt.close('all')
# plt.plot(month['maksupvm'], month['saldo'])
# plt.grid()
# plt.show()
plt.hist(monthly_incomes, bins='auto')
print(incomes.median())
display(transactions['maksupvm'].max())
display(transactions['maksupvm'].min())
# transactions.isna().sum()
(transactions['maksupvm'].max()-transactions['maksupvm'].min()).days
# plt.xscale('log')
monthly_incomes.median()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-30-252c5b73d90b>:7: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  print(incomes.median())


tilinro                    4582.00
rahamaara                   125.29
saldo                      1511.95
vientiselitekd              701.00
taplajikd                   520.00
viite                         1.00
iban_saaja                    1.00
counterparty_account_id     205.00
dtype: float64


Timestamp('2021-01-04 00:00:00')

Timestamp('2018-12-27 00:00:00')

1664.138333333334

In [110]:
def color_bar(bar_value_to_label, patches):
    min_distance = float("inf")  # initialize min_distance with infinity
    index_of_bar_to_label = 0
    for i, rectangle in enumerate(patches):  # iterate over every bar
        tmp = abs(  # tmp = distance from middle of the bar to bar_value_to_label
            (rectangle.get_x() +
                (rectangle.get_width() * (1 / 2))) - bar_value_to_label)
        if tmp < min_distance:  # we are searching for the bar with x cordinate
                                # closest to bar_value_to_label
            min_distance = tmp
            index_of_bar_to_label = i
    patches[index_of_bar_to_label].set_color('r')

In [258]:
%matplotlib widget
plt.close('all')
customer_id = 17
window_size = 100
customer = monthly_incomes[customer_id]
sorted_incomes = yearly_incomes.sort_values()
customer_ranking = sorted_incomes.index.get_loc(customer_id)
similar_customers = sorted_incomes.iloc[customer_ranking-window_size:customer_ranking+window_size].index
similar_living_costs = living_costs.loc[similar_customers]
similar_living_costs = similar_living_costs.loc[(similar_living_costs < similar_living_costs.quantile(0.99)) & (similar_living_costs > similar_living_costs.quantile(0.01))]
n, bins, patches = plt.hist(similar_living_costs, bins='auto')
color_bar(living_costs.loc[customer_id], patches)
title = 'similar_livin_'
plt.title(monthly_incomes.loc[customer_id])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, '3908.3370833333315')

In [259]:

customer_transactions = transactions.loc[transactions['tilinro'] == customer_id][['maksupvm', 'saldo']]
display(customer_transactions)
customer_transactions_weekly = customer_transactions.resample('D', on='maksupvm').saldo.mean().interpolate('time')
%matplotlib widget
plt.plot(customer_transactions['maksupvm'], customer_transactions['saldo'])
# customer_transactions_weekly.maksupvm.duplicated()
plt.plot(customer_transactions_weekly)
display(customer_transactions_weekly)
customer_transactions_weekly.isna().sum()




,maksupvm,saldo
16533,2019-01-01,14206.99
16534,2019-01-03,14032.20
16535,2019-01-04,13984.22
16536,2019-01-07,13983.93
16537,2019-01-09,13971.60
...,...,...
17738,2020-12-28,8674.40
17739,2020-12-29,8415.51
17740,2020-12-30,8402.13
17741,2020-12-30,8398.97


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

maksupvm
2019-01-01    14206.990000
2019-01-02    14119.595000
2019-01-03    14032.200000
2019-01-04    13984.220000
2019-01-05    13984.123333
                  ...     
2020-12-27     8758.156667
2020-12-28     8724.452500
2020-12-29     8415.510000
2020-12-30     8400.550000
2020-12-31     8323.630000
Freq: D, Name: saldo, Length: 731, dtype: float64

0

In [260]:
#preprocess

scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(customer_transactions_weekly.values.astype('float32').reshape(-1, 1))

# split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))



489 242


In [306]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)


# reshape into X=t and Y=t+1
look_back = 7
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
display(trainX)
display(trainY)

array([[[0.31407064, 0.31146443, 0.30885828, ..., 0.30742455,
         0.30742168, 0.30741882]],

       [[0.31146443, 0.30885828, 0.30742747, ..., 0.30742168,
         0.30741882, 0.30723494]],

       [[0.30885828, 0.30742747, 0.30742455, ..., 0.30741882,
         0.30723494, 0.30705112]],

       ...,

       [[0.14177404, 0.14110546, 0.1247533 , ..., 0.09204896,
         0.08998027, 0.0871315 ]],

       [[0.14110546, 0.1247533 , 0.10840112, ..., 0.08998027,
         0.0871315 , 0.0864805 ]],

       [[0.1247533 , 0.10840112, 0.09204896, ..., 0.0871315 ,
         0.0864805 , 0.08627026]]], dtype=float32)

array([0.30723494, 0.30705112, 0.317863  , 0.3286748 , 0.32850814,
       0.32834142, 0.3281747 , 0.3271554 , 0.32645428, 0.32575303,
       0.3250519 , 0.32492477, 0.32479763, 0.3246705 , 0.32206082,
       0.3216496 , 0.3212384 , 0.32297307, 0.32470775, 0.32644242,
       0.3281771 , 0.32744157, 0.32670605, 0.34436357, 0.36202103,
       0.3796786 , 0.39733613, 0.41499358, 0.5893441 , 0.6072864 ,
       0.6067013 , 0.6063805 , 0.6060598 , 0.60573906, 0.60541826,
       0.56658775, 0.5637235 , 0.56064504, 0.5530064 , 0.5523839 ,
       0.5517614 , 0.6002913 , 0.6478681 , 0.6408539 , 0.63383955,
       0.6268253 , 0.62628305, 0.6257409 , 0.6251986 , 0.62384456,
       0.6208707 , 0.61164826, 0.6343017 , 0.63838375, 0.64246583,
       0.64654785, 0.642735  , 0.6793097 , 0.6881768 , 0.6860915 ,
       0.7066906 , 0.7272897 , 0.74788886, 0.8890231 , 0.8873306 ,
       0.8868709 , 0.88641125, 0.8852304 , 0.8840496 , 0.88286865,
       0.8807228 , 0.87607193, 0.88537997, 0.89468807, 0.89774

In [267]:

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=10, batch_size=1, verbose=2)

Epoch 1/10
481/481 - 0s - loss: 0.0327
Epoch 2/10
481/481 - 0s - loss: 0.0060
Epoch 3/10
481/481 - 0s - loss: 0.0034
Epoch 4/10
481/481 - 0s - loss: 0.0030
Epoch 5/10
481/481 - 0s - loss: 0.0029
Epoch 6/10
481/481 - 0s - loss: 0.0027
Epoch 7/10
481/481 - 0s - loss: 0.0025
Epoch 8/10
481/481 - 0s - loss: 0.0023
Epoch 9/10
481/481 - 0s - loss: 0.0022
Epoch 10/10
481/481 - 0s - loss: 0.0019


In [268]:

# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY_real = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY_real = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY_real[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY_real[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 1504.57 RMSE
Test Score: 1097.97 RMSE


In [305]:
%matplotlib widget
plt.close('all')
# shift train predictions for plotting
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset), label='Real data')
plt.plot(trainPredictPlot, label='Train predict')
plt.plot(testPredictPlot, label='Test predict')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1]:
%matplotlib widget
plt.close('all')
# display(customer_transactions_weekly.index.values)
print(len(testPredictPlot.flatten()))
print(len(customer_transactions_weekly))
predictions = pd.DataFrame(columns=['saldo', 'maksupvm'], data={'saldo': testPredictPlot.flatten(), 'maksupvm': customer_transactions_weekly.index.values})
predictions_plot = predictions.iloc[-7:]
plt.plot(predictions_plot['maksupvm'], predictions_plot['saldo'])
plt.plot(customer_transactions_weekly.iloc[-14:])
plt.show()

NameError: name 'plt' is not defined